<a href="https://colab.research.google.com/github/keisuke0502watanabe/DTA/blob/main/xrd_20250605.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import datetime
import re
import os
import math
import copy

class XRDAnalyzer:
    def __init__(self):
        # self._ensure_dependencies()
        self.pattern_quotes = '\".+?\"'
        self.pattern_numbers = r'\d+'

    # def _ensure_dependencies(self):
    #     """Install required dependencies if not present"""
    #     try:
    #         import kaleido
    #     except ImportError:
    #         print("Installing kaleido package...")
    #         import subprocess
    #         subprocess.run(['pip', 'install', '--quiet', 'kaleido'])
    #         print("Kaleido installed successfully!")

    def extract_quoted_text(self, lines, keyword):
        """Extract text between quotes from lines containing keyword"""
        matching_lines = [s for s in lines if keyword in s]
        if not matching_lines:
            return None
        matches = re.findall(self.pattern_quotes, matching_lines[0])
        return matches[0][1:-1] if matches else None

    def load_temperature_profile(self, filename):
        """Load temperature profile data from XRD file"""
        try:
            with open(filename, encoding='shift_jis') as f:
                data = f.readlines()

            skip = data.index('*RAS_INT_START\n') + 1
            last_row = data.index('*RAS_INT_END\n')

            temp_start = float(self.extract_quoted_text(data, 'ATMOSPHERE_TEMP_START'))
            temp_end = float(self.extract_quoted_text(data, 'ATMOSPHERE_TEMP_END'))

            if self.extract_quoted_text(data, 'MEAS_COND_AXIS_UNIT-32') == "C":
                temp_start += 273.15
                temp_end += 273.15

            date_start = datetime.datetime.strptime(
                self.extract_quoted_text(data, 'MEAS_SCAN_START_TIME'),
                "%m/%d/%y %H:%M:%S"
            )
            date_end = datetime.datetime.strptime(
                self.extract_quoted_text(data, 'MEAS_SCAN_END_TIME'),
                "%m/%d/%y %H:%M:%S"
            )
            scan_speed = self.extract_quoted_text(data, 'MEAS_SCAN_SPEED')
            scan_step = self.extract_quoted_text(data, 'MEAS_SCAN_STEP')

            return {
                'skip': skip,
                'last_row': last_row,
                'temp_start': temp_start,
                'temp_end': temp_end,
                'date_start': date_start,
                'date_end': date_end,
                'scan_speed': scan_speed,
                'scan_step': scan_step
            }
        except Exception as e:
            print(f"Error loading temperature profile from {filename}: {str(e)}")
            raise

    def load_xrd_data(self, filename):
        """Load and process XRD measurement data"""
        try:
            profile = self.load_temperature_profile(filename)

            df = pd.read_csv(
                filename,
                encoding='shift_jis',
                header=None,
                skiprows=profile['skip'],
                sep=' ',
                names=['2 theta', 'Intensity / cps', 'filename']
            )

            df = df[:profile['last_row'] - profile['skip']]

            df['filename'] = filename
            df['Date_end'] = profile['date_end']
            df['Date_start'] = profile['date_start']
            df['Temp_start / K'] = round(profile['temp_start'], 0)
            df['Temp_End / K'] = round(profile['temp_end'], 0)

            df['Intensity / cps'] = (df['Intensity / cps'].astype(float) /
                             float(profile['scan_step']) *
                             float(profile['scan_speed']) / 60.0)
            df['2 theta'] = df['2 theta'].astype(float)
            df['q / nm^-1'] = 4 * math.pi / 0.154 * df['2 theta'].apply(
                lambda x: math.sin(x / 180 * math.pi / 2)
            )

            profile_df = pd.DataFrame([
                [profile['date_start'], profile['temp_start'], filename,
                 profile['scan_speed'], profile['scan_step']],
                [profile['date_end'], profile['temp_end'], filename,
                 profile['scan_speed'], profile['scan_step']]
            ],
            columns=['time', 'Temperature / K', 'filename',
                    'ScanSpeed / deg min^-1', 'ScanStep / deg.'])

            return df, profile_df

        except Exception as e:
            print(f"Error loading XRD data from {filename}: {str(e)}")
            raise

    def process_directory(self, directory, max_files=None):
        """Process XRD files in directory with progress tracking"""
        os.chdir(directory)
        file_list = [f for f in os.listdir() if f.endswith('.ras')]

        if max_files:
            file_list = file_list[:max_files]

        total_files = len(file_list)
        all_data = []
        all_profiles = []

        print(f"\nProcessing {total_files} files:")
        for i, filename in enumerate(file_list, 1):
            print(f"Processing file {i}/{total_files}: {filename}")
            try:
                df, profile = self.load_xrd_data(filename)
                all_data.append(df)
                all_profiles.append(profile)
            except Exception as e:
                print(f"Error processing {filename}: {str(e)}")
                continue

        if not all_data or not all_profiles:
            raise ValueError("No data was successfully processed")

        print("\nConcatenating data...")
        final_data = pd.concat(all_data, ignore_index=True)
        final_profiles = pd.concat(all_profiles, ignore_index=True)
        print("Data processing completed!")

        return final_data, final_profiles

    def plot_xrd_profiles(self, df, offcet=0):
        """Plot XRD profiles"""
        fig = px.line(
            df,
            x="2 theta",
            y="Intensity / cps",
            color="filename",
            title="XRD Profiles"
        )
        fig.update_layout(height=800)
        return fig

    def plot_temperature_profile(self, therm_profile):
        """Plot temperature profile with connected measurements"""
        fig = go.Figure()

        # Sort by time
        therm_profile_sorted = therm_profile.sort_values('time')

        # Add complete profile
        fig.add_trace(go.Scatter(
            x=therm_profile_sorted['time'],
            y=therm_profile_sorted['Temperature / K'],
            name='Complete Profile',
            line=dict(color='gray', width=2),
            opacity=0.5
        ))

        # Add individual measurements
        for filename in therm_profile['filename'].unique():
            mask = therm_profile['filename'] == filename
            temp_data = therm_profile[mask].sort_values('time')
            temp = temp_data['Temperature / K'].iloc[0]

            fig.add_trace(go.Scatter(
                x=temp_data['time'],
                y=temp_data['Temperature / K'],
                name=f"{temp:.0f}K",
                mode='markers+lines',
                marker=dict(size=8),
                line=dict(width=1)
            ))

        fig.update_layout(
            title="Temperature Profile",
            xaxis_title="Time",
            yaxis_title="Temperature (K)",
            height=800,
            showlegend=True,
            hovermode='x unified'
        )

        return fig

    def plot_offset_profiles(self, df, offset=5000, amp=20):
        """Plot XRD profiles with vertical offset"""
        fig = go.Figure()

        for i, filename in enumerate(df['filename'].unique()):
            df_subset = df[df['filename'] == filename]
            fig.add_trace(go.Scatter(
                x=df_subset['2 theta'],
                y=df_subset['Intensity / cps'] * amp - offset * i,
                name=filename
            ))

        fig.update_layout(
            title="Offset XRD Profiles",
            xaxis_title="2θ (degrees)",
            yaxis_title="Intensity (a.u.)",
            xaxis=dict(range=[1, 22.5]),
            height=800
        )
        return fig

    def plot_combined_view(self, df, therm_profile):
        """Create combined plot with temperature profile and XRD data"""
        fig = make_subplots(
            rows=2,
            cols=1,
            subplot_titles=("Temperature Profile", "XRD Data"),
            vertical_spacing=0.2
        )

        # Sort temperature profile by time
        therm_profile_sorted = therm_profile.sort_values('time')

        # Add main temperature profile
        fig.add_trace(
            go.Scatter(
                x=therm_profile_sorted['time'],
                y=therm_profile_sorted['Temperature / K'],
                name='Complete Profile',
                line=dict(color='gray', width=2),
                opacity=0.5
            ),
            row=1, col=1
        )

        # Add individual measurement points
        for filename in therm_profile['filename'].unique():
            mask = therm_profile['filename'] == filename
            temp_data = therm_profile[mask].sort_values('time')
            temp = temp_data['Temperature / K'].iloc[0]

            fig.add_trace(
                go.Scatter(
                    x=temp_data['time'],
                    y=temp_data['Temperature / K'],
                    name=f"{temp:.0f}K",
                    mode='markers+lines',
                    marker=dict(size=8),
                    line=dict(width=1)
                ),
                row=1, col=1
            )

        # Add XRD profiles
        for i, filename in enumerate(df['filename'].unique()):
            df_subset = df[df['filename'] == filename]
            temp = df_subset['Temp_start / K'].iloc[0]
            fig.add_trace(
                go.Scatter(
                    x=df_subset['2 theta'],
                    y=df_subset['Intensity / cps'] + 500 * i,
                    name=f"{temp:.0f}K"
                ),
                row=2, col=1
            )

        fig.update_layout(
            height=1200,
            showlegend=True,
            title_text="Temperature Profile and XRD Data",
            hovermode='x unified'
        )

        fig.update_xaxes(title_text="Time", row=1, col=1)
        fig.update_yaxes(title_text="Temperature (K)", row=1, col=1)
        fig.update_xaxes(title_text="2θ (degrees)", row=2, col=1)
        fig.update_yaxes(title_text="Intensity (a.u.)", row=2, col=1)

        return fig

    def plot_and_display(self, data, profiles, save_html=False, filename_base=None):
        """Generate and display all plots, optionally save as HTML"""
        plots = []

        print("\nGenerating and displaying plots...")

        print("\n1. XRD Profiles")
        xrd_plot = self.plot_xrd_profiles(data)
        xrd_plot.show()
        plots.append(xrd_plot)

        print("\n2. Temperature Profile")
        temp_plot = self.plot_temperature_profile(profiles)
        temp_plot.show()
        plots.append(temp_plot)

        print("\n3. Offset XRD Profiles")
        offset_plot = self.plot_offset_profiles(data)
        offset_plot.show()
        plots.append(offset_plot)

        print("\n4. Combined View")
        combined_plot = self.plot_combined_view(data, profiles)
        combined_plot.show()
        plots.append(combined_plot)

        if save_html and filename_base:
            print("\nSaving plots to HTML...")
            os.makedirs("html", exist_ok=True)
            for i, fig in enumerate(plots, 1):
                suffix = f"_{i}" if i > 1 else ""
                html_path = f"html/{filename_base}{suffix}.html"
                fig.write_html(html_path)
                print(f"Saved figure {i} to {html_path}")

In [2]:
#解析用１　結晶子サイズ
import numpy as np
from scipy.signal import find_peaks, peak_widths
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def analyze_xrd_peaks(two_theta, intensity, min_prominence=10, k=0.9, wavelength=0.154):
    """
    XRDパターンからピークを検出し、結晶サイズを計算する関数
    """
    # ピークの検出（より細かいパラメータ制御）
    peaks, properties = find_peaks(intensity,
                                 prominence=min_prominence,  # より小さな prominence
                                 width=2,                   # より狭いピーク幅も検出
                                 distance=10,               # ピーク間の最小距離
                                 height=max(intensity)*0.01) # 最大強度の1%以上のピークを検出

    # ピーク幅（FWHM）の計算
    widths_result = peak_widths(intensity, peaks, rel_height=0.5)
    fwhm_idx = widths_result[0]  # FWHM in index units

    # 結果を格納するリスト
    peak_info = []

    for i, peak_idx in enumerate(peaks):
        theta = two_theta[peak_idx]
        d_spacing = wavelength / (2 * np.sin(np.radians(theta/2)))
        fwhm_2theta = fwhm_idx[i] * (two_theta[1] - two_theta[0])

        beta = np.radians(fwhm_2theta)
        theta_rad = np.radians(theta/2)

        if beta > 0:
            crystallite_size = (k * wavelength) / (beta * np.cos(theta_rad))
        else:
            crystallite_size = float('inf')

        peak_info.append({
            'two_theta': theta,
            'd_spacing': d_spacing,
            'intensity': intensity[peak_idx],
            'fwhm': fwhm_2theta,
            'crystallite_size': crystallite_size
        })

    return peak_info

def plot_xrd_analysis_plotly(two_theta, intensity, peak_info):
    """
    Plotlyを使用してXRDパターンと結晶サイズを可視化する関数
    """
    # サブプロットの作成
    fig = make_subplots(rows=2, cols=1,
                        subplot_titles=('XRD Pattern with Detected Peaks',
                                      'Crystallite Size Distribution'),
                        vertical_spacing=0.2,
                        row_heights=[0.7, 0.3])

    # XRDパターンのプロット
    fig.add_trace(
        go.Scatter(x=two_theta,
                  y=intensity,
                  name='XRD Pattern',
                  line=dict(color='blue'),
                  hovertemplate='2θ: %{x:.2f}°<br>Intensity: %{y:.0f}<extra></extra>'),
        row=1, col=1
    )

    # 検出されたピークのプロット
    peak_positions = [p['two_theta'] for p in peak_info]
    peak_intensities = [p['intensity'] for p in peak_info]
    d_spacings = [p['d_spacing'] for p in peak_info]
    crystallite_sizes = [p['crystallite_size'] for p in peak_info]
    fwhm_values = [p['fwhm'] for p in peak_info]

    # ピークポイントの追加
    fig.add_trace(
        go.Scatter(x=peak_positions,
                  y=peak_intensities,
                  mode='markers',
                  name='Peaks',
                  marker=dict(color='red', size=8),
                  hovertemplate='2θ: %{x:.2f}°<br>' +
                              'Intensity: %{y:.0f}<br>' +
                              'd-spacing: %{customdata[0]:.3f} nm<br>' +
                              'FWHM: %{customdata[1]:.3f}°<br>' +
                              'Crystallite Size: %{customdata[2]:.1f} nm' +
                              '<extra></extra>',
                  customdata=np.array([d_spacings, fwhm_values, crystallite_sizes]).T),
        row=1, col=1
    )

    # 結晶サイズ分布のプロット
    fig.add_trace(
        go.Bar(x=peak_positions,
               y=crystallite_sizes,
               name='Crystallite Size',
               marker_color='rgb(158,202,225)',
               hovertemplate='2θ: %{x:.2f}°<br>' +
                           'Crystallite Size: %{y:.1f} nm<br>' +
                           'd-spacing: %{customdata:.3f} nm' +
                           '<extra></extra>',
               customdata=d_spacings),
        row=2, col=1
    )

    # レイアウトの設定
    fig.update_layout(
        title_text='XRD Analysis Results',
        showlegend=True,
        height=800,
        width=1000,
        hovermode='closest'
    )

    # X軸のラベル設定
    fig.update_xaxes(title_text='2θ (degrees)', row=1, col=1)
    fig.update_xaxes(title_text='2θ (degrees)', row=2, col=1)

    # Y軸のラベル設定
    fig.update_yaxes(title_text='Intensity (cps)', row=1, col=1)
    fig.update_yaxes(title_text='Crystallite Size (nm)', row=2, col=1)

    return fig

# 使用例:
# データの読み込み
# two_theta = your_two_theta_data
# intensity = your_intensity_data

# ピーク解析の実行
# peak_info = analyze_xrd_peaks(two_theta, intensity)

# Plotlyでの可視化
# fig = plot_xrd_analysis_plotly(two_theta, intensity, peak_info)
# fig.show()  # ブラウザで表示

In [3]:
#解析用2　Williamson-Hallプロット
import numpy as np
from scipy.signal import find_peaks, peak_widths
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import linregress

def analyze_xrd_peaks_wh(two_theta, intensity, min_prominence=10, k=0.9, wavelength=0.154):
    """
    XRDパターンからピークを検出し、Williamson-Hall解析を行う関数
    """
    # ピークの検出
    peaks, properties = find_peaks(intensity,
                                 prominence=min_prominence,
                                 width=2,
                                 distance=10,
                                 height=max(intensity)*0.01)

    # ピーク幅（FWHM）の計算
    widths_result = peak_widths(intensity, peaks, rel_height=0.5)
    fwhm_idx = widths_result[0]

    peak_info = []

    for i, peak_idx in enumerate(peaks):
        # 基本的なピーク情報の計算
        theta = two_theta[peak_idx]
        d_spacing = wavelength / (2 * np.sin(np.radians(theta/2)))
        fwhm_2theta = fwhm_idx[i] * (two_theta[1] - two_theta[0])

        # Williamson-Hall解析用のパラメータ計算
        beta_rad = np.radians(fwhm_2theta)  # FWHM in radians
        theta_rad = np.radians(theta/2)

        # β*cos(θ) vs 4*sin(θ) for W-H plot
        beta_cos_theta = beta_rad * np.cos(theta_rad)
        four_sin_theta = 4 * np.sin(theta_rad)

        if beta_rad > 0:
            crystallite_size = (k * wavelength) / (beta_rad * np.cos(theta_rad))
        else:
            crystallite_size = float('inf')

        peak_info.append({
            'two_theta': theta,
            'd_spacing': d_spacing,
            'intensity': intensity[peak_idx],
            'fwhm': fwhm_2theta,
            'crystallite_size': crystallite_size,
            'beta_cos_theta': beta_cos_theta,
            'four_sin_theta': four_sin_theta
        })

    return peak_info

def calculate_wh_parameters(peak_info):
    """
    Williamson-Hall解析からひずみと結晶子サイズを計算
    """
    x_values = [p['four_sin_theta'] for p in peak_info]
    y_values = [p['beta_cos_theta'] for p in peak_info]

    # 線形回帰による解析
    slope, intercept, r_value, p_value, std_err = linregress(x_values, y_values)

    # パラメータの計算
    strain = slope / 4  # ひずみ
    crystallite_size = 0.9 * 0.154 / intercept if intercept > 0 else float('inf')  # nm

    return {
        'strain': strain,
        'crystallite_size': crystallite_size,
        'r_squared': r_value**2,
        'slope': slope,
        'intercept': intercept
    }

def plot_xrd_analysis_with_wh(two_theta, intensity, peak_info):
    """
    XRDパターンとWilliamson-Hallプロットを表示する関数
    """
    # W-Hプロットのデータ準備
    wh_params = calculate_wh_parameters(peak_info)
    x_wh = [p['four_sin_theta'] for p in peak_info]
    y_wh = [p['beta_cos_theta'] for p in peak_info]

    # 回帰直線のデータ点
    x_fit = np.linspace(min(x_wh), max(x_wh), 100)
    y_fit = wh_params['slope'] * x_fit + wh_params['intercept']

    # サブプロットの作成（3つのグラフ）
    fig = make_subplots(rows=2, cols=2,
                       subplot_titles=('XRD Pattern with Detected Peaks',
                                     'Williamson-Hall Plot',
                                     'Crystallite Size Distribution'),
                       vertical_spacing=0.15,
                       horizontal_spacing=0.1,
                       specs=[[{}, {}],
                             [{"colspan": 2}, None]])

    # 1. XRDパターンのプロット
    fig.add_trace(
        go.Scatter(x=two_theta,
                  y=intensity,
                  name='XRD Pattern',
                  line=dict(color='blue'),
                  hovertemplate='2θ: %{x:.2f}°<br>Intensity: %{y:.0f}<extra></extra>'),
        row=1, col=1
    )

    # ピークのプロット
    peak_positions = [p['two_theta'] for p in peak_info]
    peak_intensities = [p['intensity'] for p in peak_info]
    d_spacings = [p['d_spacing'] for p in peak_info]
    crystallite_sizes = [p['crystallite_size'] for p in peak_info]
    fwhm_values = [p['fwhm'] for p in peak_info]

    fig.add_trace(
        go.Scatter(x=peak_positions,
                  y=peak_intensities,
                  mode='markers',
                  name='Peaks',
                  marker=dict(color='red', size=8),
                  hovertemplate='2θ: %{x:.2f}°<br>' +
                              'Intensity: %{y:.0f}<br>' +
                              'd-spacing: %{customdata[0]:.3f} nm<br>' +
                              'FWHM: %{customdata[1]:.3f}°<br>' +
                              'Crystallite Size: %{customdata[2]:.1f} nm' +
                              '<extra></extra>',
                  customdata=np.array([d_spacings, fwhm_values, crystallite_sizes]).T),
        row=1, col=1
    )

    # 2. Williamson-Hallプロット
    fig.add_trace(
        go.Scatter(x=x_wh,
                  y=y_wh,
                  mode='markers',
                  name='W-H Data',
                  marker=dict(color='green', size=8),
                  hovertemplate='4sinθ: %{x:.4f}<br>' +
                              'βcosθ: %{y:.4f}<extra></extra>'),
        row=1, col=2
    )

    # W-H回帰直線
    fig.add_trace(
        go.Scatter(x=x_fit,
                  y=y_fit,
                  mode='lines',
                  name='W-H Fit',
                  line=dict(color='red', dash='dash'),
                  hovertemplate='4sinθ: %{x:.4f}<br>' +
                              'βcosθ: %{y:.4f}<extra></extra>'),
        row=1, col=2
    )

    # 3. 結晶サイズ分布
    fig.add_trace(
        go.Bar(x=peak_positions,
               y=crystallite_sizes,
               name='Crystallite Size',
               marker_color='rgb(158,202,225)',
               hovertemplate='2θ: %{x:.2f}°<br>' +
                           'Crystallite Size: %{y:.1f} nm<br>' +
                           'd-spacing: %{customdata:.3f} nm' +
                           '<extra></extra>',
               customdata=d_spacings),
        row=2, col=1
    )

    # レイアウトの更新
    fig.update_layout(
        title_text=f'XRD Analysis with Williamson-Hall Plot<br>' +
                  f'Average Crystallite Size: {wh_params["crystallite_size"]:.1f} nm, ' +
                  f'Strain: {wh_params["strain"]:.6f}, ' +
                  f'R²: {wh_params["r_squared"]:.3f}',
        showlegend=True,
        height=1000,
        width=1200,
        hovermode='closest'
    )

    # 軸ラベルの設定
    fig.update_xaxes(title_text='2θ (degrees)', row=1, col=1)
    fig.update_xaxes(title_text='4sinθ', row=1, col=2)
    fig.update_xaxes(title_text='2θ (degrees)', row=2, col=1)

    fig.update_yaxes(title_text='Intensity (cps)', row=1, col=1)
    fig.update_yaxes(title_text='βcosθ', row=1, col=2)
    fig.update_yaxes(title_text='Crystallite Size (nm)', row=2, col=1)

    return fig

# 使用例:
# peak_info = analyze_xrd_peaks_wh(two_theta, intensity)
# fig = plot_xrd_analysis_with_wh(two_theta, intensity, peak_info)
# fig.show()

In [5]:
from google.colab import drive
drive.mount('/content/drive')

analyzer = XRDAnalyzer()
path = '/content/drive/MyDrive/Data/watanabe/xrd/2025/2025/ras'
data, profiles = analyzer.process_directory(path)

# デフォルト値を使用
# analyzer.plot_and_display(data, profiles)

# オフセットを調整して表示
# analyzer.plot_and_display(data, profiles,
                        #  offset_2theta={'offset': 25000, 'amp': 10},  # 2θプロットのオフセットを小さく
                        #  offset_q={'offset': 30000, 'amp': 15}

                          # )       # Qプロットは別の値に

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Processing 17 files:
Processing file 1/17: trilaurin-25c.ras
Processing file 2/17: 1-trilaurin-25c.ras
Processing file 3/17: 2-trilaurin-60c.ras
Processing file 4/17: 2-trilaurin-60c-2nd.ras
Processing file 5/17: 3-trilaurin-80c.ras
Processing file 6/17: 3-trilaurin-80c-2nd.ras
Processing file 7/17: 4-trilaurin-42c.ras
Processing file 8/17: 5-trilaurin-42c.ras
Processing file 9/17: 6-trilaurin-42c.ras
Processing file 10/17: 7-trilaurin-60c.ras
Processing file 11/17: 8-trilaurin-80c.ras
Processing file 12/17: 9-trilaurin-80c.ras
Processing file 13/17: 10-trilaurin-55c.ras
Processing file 14/17: 11-trilaurin-55c.ras
Processing file 15/17: 12-trilaurin-42c.ras
Processing file 16/17: 13-trilaurin-42c.ras
Processing file 17/17: 14-trilaurin-60c.ras

Concatenating data...
Data processing completed!


In [6]:
profiles

,time,Temperature / K,filename,ScanSpeed / deg min^-1,ScanStep / deg.
0,2025-06-05 13:49:27,298.15,trilaurin-25c.ras,4.000000,0.0200000000
1,2025-06-05 13:59:06,298.15,trilaurin-25c.ras,4.000000,0.0200000000
2,2025-06-05 14:23:51,298.15,1-trilaurin-25c.ras,4.000000,0.0200000000
3,2025-06-05 14:29:49,298.15,1-trilaurin-25c.ras,4.000000,0.0200000000
4,2025-06-05 14:33:18,333.85,2-trilaurin-60c.ras,4.000000,0.0200000000
5,2025-06-05 14:39:16,333.15,2-trilaurin-60c.ras,4.000000,0.0200000000
6,2025-06-05 14:39:41,333.15,2-trilaurin-60c-2nd.ras,4.000000,0.0200000000
7,2025-06-05 14:45:38,333.15,2-trilaurin-60c-2nd.ras,4.000000,0.0200000000
8,2025-06-05 14:48:07,353.95,3-trilaurin-80c.ras,4.000000,0.0200000000
9,2025-06-05 14:54:04,353.15,3-trilaurin-80c.ras,4.000000,0.0200000000


In [7]:
def plot5(data, list):
  fig = go.Figure()

  # オフセット値の設定
  offset = -500  # 必要に応じて調整
  text_offset = 200  # テキストの上方向へのオフセット
  # min_intensity = data['Intensity / cps'].min()  # 最大強度を取得

  # データを下から順に表示
  for i, filename in enumerate(reversed(list)):  # reversed で順序を逆にする
      df_temp = data[data['filename'] == filename]
      base_offset = offset * i

      fig.add_trace(
          go.Scatter(
              x=df_temp['2 theta'],
              y=df_temp['Intensity / cps'] + offset * i,  # 下から順にオフセットを加える
              name=filename,
          )
      )

      # ファイル名をテキストとして追加（プロットの上部に）
      min_intensity = df_temp['Intensity / cps'].min()
      # fig.add_annotation(
      #     x=40,  # 最小強度の位置のx座標
      #     y=min_intensity + base_offset + text_offset,
      #     text=filename,
      #     showarrow=False,
      #     font=dict(size=12),
      #     xanchor='left'
      # )

  fig.update_layout(
      title="XRD Patterns (Stacked)",
      xaxis_title="2θ (degrees)",
      yaxis_title="Intensity (cps)",
      height=800,  # グラフの高さを調整
      showlegend=True
  )

  return fig
  # fig.show()

# fig1 = copy.deepcopy(fig)

In [8]:
#一つだけプロットしたい時

df={}
#data からdata['filename']に固有な値を抽出する
data['filename'].unique()
#'filename'の列の値が、'1-N4441TFSI-25C.ras'である行のみを抽出して、それを新たなデータフレームとして設定
df[0]=data[data['filename'] == '1-N4441TFSI-25C.ras']
# data[data['filename'] == '1-N4441TFSI-25C.ras']
df[0]
fig = go.Figure()

fig.add_trace(
            go.Scatter(
                x=df[0]['2 theta'],
                y=df[0]['Intensity / cps'],
                name=df[0]['filename'][0],
        )
            )
# レイアウトの設定を追加
fig.update_layout(
    title="XRD Pattern",
    xaxis_title="2θ (degrees)",
    yaxis_title="Intensity (cps)",
    height=600,
    showlegend=True,
    template='plotly'  # プロットのテンプレート指定
)

fig.show()


# data['filename'].unique()

KeyError: 0

In [9]:
#全体を一気に見たい時
fig = go.Figure()

offset = -500  # オフセット値（必要に応じて調整）

for i, filename in enumerate(data['filename'].unique()):
    df[i] = data[data['filename'] == filename]

    fig.add_trace(
        go.Scatter(
            x=df[i]['2 theta'],
            y=df[i]['Intensity / cps'] + offset * i,  # オフセットを追加
            name=filename,
        )
    )

fig.update_layout(
    title="XRD Patterns (Offset)",
    xaxis_title="2θ (degrees)",
    yaxis_title="Intensity (cps)",
    height=600,
    showlegend=True
)

fig.show()

In [ ]:
#５個づつプロットしたい時
import copy

# 全体をグループ化してプロット
for i in range(0, 15, 5):
    # list()を使わずに直接スライスを使用
    file_list = data['filename'].unique()[i:i+5].tolist()
    # または
    # file_list = data['filename'].unique()[i:i+5]

    fig = plot5(data, file_list)
    fig_copy = copy.deepcopy(fig)
    fig_copy.show()

In [ ]:
# 特定のファイルを番号で抽出してプロット
print(data['filename'].unique())
num_list1=[3,10,13]
list1=[]
for num in num_list1:
  list1.append(data['filename'].unique()[num])
fig=plot5(data, list1)
fig1 = copy.deepcopy(fig)
fig1.show()

['1-N4441TFSI-25C.ras' '2-N4441TFSI--20C.ras' '3-N4441TFSI--56C.ras'
 '4-N4441TFSI--82C.ras' '5-N4441TFSI--56C-1st.ras'
 '6-N4441TFSI--56C-2nd.ras' '7-N4441TFSI--30C-1st.ras'
 '8-N4441TFSI--30C-2nd.ras' '9-N4441TFSI--20C-1st.ras'
 '10-N4441TFSI--20C-2nd.ras' '11-N4441TFSI--6C-1st.ras'
 '12-N4441TFSI--6C-2nd.ras' '13-N4441TFSI-13C-1st.ras'
 '14-N4441TFSI-13C-2nd.ras' '15-N4441TFSI-35C-1st.ras'
 '16-N4441TFSI-35C-2nd.ras' '15-N4441TFSI-25C.ras']


In [ ]:
# 7番目のファイルのデータを取得
df_temp = data[data['filename'] == data['filename'].unique()[7]]

# import os
print(os.getcwd())
# または UNIX/Mac形式のパスの場合
# df_temp.to_csv('/Users/YourName/Desktop/filename.csv', index=False)
# CSVとして出力
# df_temp.to_csv('filename.csv', index=False)

#wh
# 使用例:
peak_info = analyze_xrd_peaks_wh(two_theta, intensity, min_prominence=100)

# パラメータ計算
wh_params = calculate_wh_parameters(peak_info)
fig = plot_xrd_analysis_with_wh(two_theta, intensity, peak_info)
fig.show()

In [ ]:
#wh
# 使用例:
peak_info = analyze_xrd_peaks_wh(two_theta, intensity, min_prominence=100)

# パラメータ計算
wh_params = calculate_wh_parameters(peak_info)
fig = plot_xrd_analysis_with_wh(two_theta, intensity, peak_info)
fig.show()

In [ ]:
# 特定のファイルを番号で抽出してプロット
print(data['filename'].unique())
num_list1=[13]
list1=[]
for num in num_list1:
  list1.append(data['filename'].unique()[num])
fig=plot5(data, list1)
fig1 = copy.deepcopy(fig)
fig1.show()

['1-N4441TFSI-25C.ras' '2-N4441TFSI--20C.ras' '3-N4441TFSI--56C.ras'
 '4-N4441TFSI--82C.ras' '5-N4441TFSI--56C-1st.ras'
 '6-N4441TFSI--56C-2nd.ras' '7-N4441TFSI--30C-1st.ras'
 '8-N4441TFSI--30C-2nd.ras' '9-N4441TFSI--20C-1st.ras'
 '10-N4441TFSI--20C-2nd.ras' '11-N4441TFSI--6C-1st.ras'
 '12-N4441TFSI--6C-2nd.ras' '13-N4441TFSI-13C-1st.ras'
 '14-N4441TFSI-13C-2nd.ras' '15-N4441TFSI-35C-1st.ras'
 '16-N4441TFSI-35C-2nd.ras' '15-N4441TFSI-25C.ras']


In [ ]:
# 7番目のファイルのデータを取得
df_temp = data[data['filename'] == data['filename'].unique()[13]]

two_theta = df_temp['2 theta']
intensity = df_temp['Intensity / cps']
# import os
# print(os.getcwd())
# または UNIX/Mac形式のパスの場合
# df_temp.to_csv('/Users/YourName/Desktop/filename.csv', index=False)
# CSVとして出力
# df_temp.to_csv('filename.csv', index=False)

#wh
# 使用例:
peak_info = analyze_xrd_peaks_wh(two_theta, intensity, min_prominence=100)

# パラメータ計算
wh_params = calculate_wh_parameters(peak_info)
fig = plot_xrd_analysis_with_wh(two_theta, intensity, peak_info)
fig.show()

In [ ]:
import numpy as np
from scipy.signal import find_peaks, peak_widths
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import linregress

def analyze_xrd_peaks_wh(two_theta, intensity, min_prominence=10, k=0.9, wavelength=0.154):
    """
    XRDパターンからピークを検出し、Williamson-Hall解析を行う関数
    """
    # データを numpy 配列に変換
    two_theta = np.array(two_theta)
    intensity = np.array(intensity)

    # ピークの検出（インデックスの範囲チェック付き）
    peaks, properties = find_peaks(intensity,
                                 prominence=min_prominence,
                                 width=2,
                                 distance=10,
                                 height=max(intensity)*0.01)

    # インデックスの範囲チェック
    peaks = peaks[peaks < len(two_theta)]

    # ピーク幅（FWHM）の計算
    widths_result = peak_widths(intensity, peaks, rel_height=0.5)
    fwhm_idx = widths_result[0]

    peak_info = []

    for i, peak_idx in enumerate(peaks):
        if peak_idx >= len(two_theta):
            continue

        # 基本的なピーク情報の計算
        theta = two_theta[peak_idx]
        d_spacing = wavelength / (2 * np.sin(np.radians(theta/2)))
        fwhm_2theta = fwhm_idx[i] * (two_theta[1] - two_theta[0])

        # Williamson-Hall解析用のパラメータ計算
        beta_rad = np.radians(fwhm_2theta)
        theta_rad = np.radians(theta/2)

        # β*cos(θ) vs 4*sin(θ) for W-H plot
        beta_cos_theta = beta_rad * np.cos(theta_rad)
        four_sin_theta = 4 * np.sin(theta_rad)

        if beta_rad > 0:
            crystallite_size = (k * wavelength) / (beta_rad * np.cos(theta_rad))
        else:
            crystallite_size = float('inf')

        peak_info.append({
            'two_theta': theta,
            'd_spacing': d_spacing,
            'intensity': intensity[peak_idx],
            'fwhm': fwhm_2theta,
            'crystallite_size': crystallite_size,
            'beta_cos_theta': beta_cos_theta,
            'four_sin_theta': four_sin_theta
        })

    return peak_info

def calculate_wh_parameters(peak_info):
    """
    Williamson-Hall解析からひずみと結晶子サイズを計算
    """
    if not peak_info:
        return {
            'strain': 0,
            'crystallite_size': 0,
            'r_squared': 0,
            'slope': 0,
            'intercept': 0
        }

    x_values = [p['four_sin_theta'] for p in peak_info]
    y_values = [p['beta_cos_theta'] for p in peak_info]

    # データが十分にある場合のみ線形回帰を実行
    if len(x_values) > 1:
        slope, intercept, r_value, p_value, std_err = linregress(x_values, y_values)
        strain = slope / 4
        crystallite_size = 0.9 * 0.154 / intercept if intercept > 0 else float('inf')
        r_squared = r_value**2
    else:
        strain = 0
        crystallite_size = 0
        r_squared = 0
        slope = 0
        intercept = 0

    return {
        'strain': strain,
        'crystallite_size': crystallite_size,
        'r_squared': r_squared,
        'slope': slope,
        'intercept': intercept
    }

# プロット関数は以前と同じ

In [ ]:
import numpy as np
from scipy.signal import find_peaks, savgol_filter
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

def extract_halo_peaks(two_theta, intensity, window_length=15, polyorder=3, prominence=50):
    """
    XRDデータからハローピークを抽出する関数

    Parameters:
    -----------
    two_theta : array-like
        2θ値の配列
    intensity : array-like
        強度の配列
    window_length : int
        Savitzky-Golayフィルタのウィンドウ長
    polyorder : int
        Savitzky-Golayフィルタの多項式次数
    prominence : float
        ピーク検出の際のprominence値

    Returns:
    --------
    tuple
        (ハローピークの2θ位置, ハローピークの強度, 平滑化されたデータ)
    """
    # データの平滑化
    smoothed = savgol_filter(intensity, window_length, polyorder)

    # バックグラウンドの推定と除去
    background = savgol_filter(smoothed, window_length*3, polyorder)
    corrected = smoothed - background

    # ハローピークの検出
    peaks, properties = find_peaks(corrected,
                                 prominence=prominence,
                                 width=5,
                                 distance=20)

    # ガウス関数でのフィッティング
    def gaussian(x, amplitude, center, width):
        return amplitude * np.exp(-(x - center)**2 / (2 * width**2))

    halo_peaks = []
    for peak in peaks:
        try:
            # ピーク周辺のデータを切り出し
            window = slice(max(0, peak-20), min(len(two_theta), peak+20))
            x_fit = two_theta[window]
            y_fit = corrected[window]

            # 初期パラメータの推定
            p0 = [corrected[peak], two_theta[peak], 2.0]

            # フィッティング
            popt, _ = curve_fit(gaussian, x_fit, y_fit, p0=p0)

            if 10 < popt[1] < 40:  # 一般的なハローピークの2θ範囲
                halo_peaks.append((popt[1], popt[0]))
        except:
            continue

    return np.array(halo_peaks)[:, 0], np.array(halo_peaks)[:, 1], smoothed

def plot_results(two_theta, intensity, halo_positions, halo_intensities, smoothed):
    """結果をプロットする関数"""
    plt.figure(figsize=(10, 6))
    plt.plot(two_theta, intensity, 'b-', alpha=0.5, label='Original')
    plt.plot(two_theta, smoothed, 'r-', label='Smoothed')
    plt.plot(halo_positions, halo_intensities, 'go', label='Halo peaks')

    plt.xlabel('2θ (degrees)')
    plt.ylabel('Intensity (cps)')
    plt.title('XRD Pattern with Detected Halo Peaks')
    plt.legend()
    plt.grid(True)
    plt.show()

